In [1]:
import os
import requests
from pymongo import MongoClient
import pandas as pd
import random
from IPython.display import display

cde_pth =os.getcwd()
print("Code Path ::"+ cde_pth)

DAILYMED_URL = "https://dailymed.nlm.nih.gov/dailymed/services/v2"

DB_HOST = "localhost"

DB_PORT = 27017

DB_NAME ="dailymed"

NDC_COLXN ="spls"
NDC_COLXN_NEXT ="spls_next"


# Styling notebook
from IPython.core.display import HTML
# Styling notebook
def css_styling():
    styles = open( cde_pth + "/custom.css", "r").read()
    HTML(''.format(styles))
    #return HTML(styles)
css_styling()

HTML("""
<style>
.renderjson a              { text-decoration: none !important; }
.renderjson .disclosure    { color: crimson;
                             font-size: 100%; }
.renderjson .syntax        { color: grey; }
.renderjson .string        { color: red; }
.renderjson .number        { color: cyan; }
.renderjson .boolean       { color: plum; }
.renderjson .key           { color: lightblue; }
.renderjson .keyword       { color: lightgoldenrodyellow; }
.renderjson .object.syntax { color: lightseagreen; }
.renderjson .array.syntax  { color: lightsalmon; }
/* unvisited link */

.renderjson a:link {
  color: blue;
}

/* visited link */
.renderjson a:visited {
  color: green;
}

/* mouse over link */
.renderjson a:hover {
  color: hotpink;
}

/* selected link */
.renderjson a:active {
  color: blue;
}
</style>
""")
#----------------------------------------------------------    
# Class utilized for Display At Notebook
'''
    Class utilized for Display At Notebook
    https://stackoverflow.com/questions/18873066/pretty-json-formatting-in-ipython-notebook
    https://github.com/caldwell/renderjson
'''
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):        
        display_html('<div id="{}" style="height: 250px; width:100%;" class="renderjson"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_icons("+", "-");
          renderjson.set_show_to_level(1);
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True) 


import json
from datetime import datetime
from typing import Any

from bson import ObjectId


class MongoJSONEncoder(json.JSONEncoder):
    def default(self, o: Any) -> Any:
        if isinstance(o, ObjectId):
            return str(o)
        if isinstance(o, datetime):
            return str(o)
        return json.JSONEncoder.default(self, o)
    
class objdict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)
            

class MongoDBClient:
    def __init__(self, db_hostname=None,db_port=None, db_name=None):
        if db_hostname is None:
            self.hostname =DB_HOST
        else:
            self.hostname = db_hostname
        if db_port is None:
            self.port = DB_PORT
        else:
            self.port = db_port
        if db_name is None:
            self.dbName = DB_NAME
        else:
            self.dbName = db_name
    
    def create_client(self):
        client = MongoClient(self.hostname,self.port)        
        self.client = client
        return client
    
    def get_list_databases(self):
        if self.client is None:
            self.client = self.create_client()
            
        return self.client.list_databases()
        
    def get_dailymed_db(self,dbName=None):
        client = self.create_client()
        db = None
        if dbName is None:
            db = client[self.dbName]
        else:
            db = client[dbName]
        return db
    
    def get_list_collection(self, dbName=None):
        if self.client is None:
            self.client = self.create_client()
            
        db = self.get_dailymed_db(dbName)
        clxns = None
        if db is not None:
            clxns = db.list_collection_names()
        return clxns
            
class BaseRequest:
    def __init__(self,api_url=None):
        self.api_url = api_url
    
    def get_request(self,api_url=None,path_suffix=None, params=None):
        if api_url is None:
            api_url = self.api_url
            
        if path_suffix is not None:
            api_url = api_url+"/" + path_suffix
        print("Fetching URL : "+api_url)
        r = requests.get(url = api_url, params = params)
        return r.json()
    


Code Path ::/Users/anand/Documents/sayansi/workspaces/condaws/dailymed_sracping/notebook


In [2]:
def testClasses():
    print("~~~~~~~~~~~ Test Run Classes START ~~~~~~~~~~~~~~~")
    br = BaseRequest(api_url=DAILYMED_URL)
    #print(br.api_url)
    respo = br.get_request(None,"spls.json",{})
   

    mClient = MongoDBClient()
    db = mClient.create_client()
    '''
    dbl = mClient.get_list_databases()
    for db in dbl:
        print(db)

    dbclxns = mClient.get_list_collection(DB_NAME)
    '''
    print(db[DB_NAME][NDC_COLXN])
    #dbclxns['spls'].insert_many(respo["data"])
    #db[DB_NAME][NDC_COLXN].insert_many(respo["data"])
    new_entry = MongoJSONEncoder().encode(respo["metadata"])
   
    if respo["metadata"]['previous_page'] == "null":
        respo["metadata"]['previous_page']= None
    if respo["metadata"]['previous_page_url'] == "null":
        respo["metadata"]['previous_page_url']= None
    #db[DB_NAME][NDC_COLXN_NEXT].insert_one(respo["metadata"])
    
    return respo
    print("~~~~~~~~~~~ Test Run Classes END ~~~~~~~~~~~~~~~")

respo = testClasses()
RenderJSON(respo)
df = pd.DataFrame.from_dict(respo["data"])
df.head(10)

~~~~~~~~~~~ Test Run Classes START ~~~~~~~~~~~~~~~
Fetching URL : https://dailymed.nlm.nih.gov/dailymed/services/v2/spls.json
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'dailymed'), 'spls')


,spl_version,published_date,title,setid
0,1,"May 12, 2023","ARIELLA NAIL REPAIR PEN (POLIGLUSAM, SALICYLIC...",069c1bf6-9df0-4e86-9928-f81cb59244c3
1,1,"May 12, 2023",ALPRAZOLAM TABLET [REMEDYREPACK INC.],0ff1b675-08c0-45bf-90fa-16232d44d9e9
2,1,"May 12, 2023",TRIAZOLAM TABLET [A-S MEDICATION SOLUTIONS],5a330f08-5d8f-4f8d-8847-a45b734412b4
3,1,"May 12, 2023",PHENOXYBENZAMINE HYDROCHLORIDE CAPSULE [AUROBI...,75f26681-a079-4ff3-9fff-75526041374c
4,1,"May 12, 2023",DRX CHOICE CHILDRENS STUFFY NOSE AND CHEST CON...,7a3bb739-ff50-4d29-b270-d6b084906493
5,1,"May 12, 2023",DRX CHOICE MAXIMUM STRENGTH SEVERE COUGH PLUS ...,a096cb76-0d1c-4ca8-854a-5199da0bc8f7
6,1,"May 12, 2023",DAYTIME NIGHTTIME COLD FLU RELIEF (ACETAMINOPH...,f3d9d25b-486d-c693-e053-2a95a90a533b
7,1,"May 12, 2023",PRAZOSIN HCL CAPSULE [DIRECT_RX],fb6db16d-3768-631b-e053-6294a90a1cee
8,2,"May 12, 2023","CEFEPIME INJECTION, POWDER, FOR SOLUTION [APOT...",02aa374f-37b4-456a-b5de-cfd3bbb6ce6e
9,2,"May 12, 2023",PREFERRED STOOL SOFTENER (DOCUSATE SODIUM) CAP...,217d13a0-3205-492c-af49-82eb80dba53a


In [4]:
from bs4 import BeautifulSoup
import json

#Load xml
xml_parser = BeautifulSoup(open('car.xml'), 'xml')

fspec = BeautifulSoup(open('fspc.json'), 'html.parser')
#fspec_json=json.loads(fspec.text)
jsn ={}
with open(cde_pth+"/fspc.json","r") as f:
    contents = f.read()
    jsn = json.loads(contents)
    #print(contents)
RenderJSON(jsn)


In [ ]:
#print(xml_parser.prettify())

In [5]:
class JsonSchemaUtil:
    def __init__(self , file_location=None,file_name=None , schema_json=None):
        
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.file_name = file_name
        
        self.schema_json = schema_json if schema_json is not None else self.load_json(self.file_location, self.file_name )
        #print("------ JsonSchemaUtil initialized with")
        #print(self.file_location)
        #print(self.file_name)    
        #print(self.schema_json)    
        #print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        
    def load_json(self,file_location=None,file_name=None):
        jsn ={}
        with open(file_location+"/"+file_name,"r") as f:
            contents = f.read()
            jsn = json.loads(contents)
        return jsn
    
    def get_object_type(self,schema_json=None):
        if schema_json is None:
            schema_json = self.schema_json
        if "type" in schema_json and schema_json["type"]=="object":
            return "O"
        elif "type" in schema_json and schema_json["type"]=="array":
            return "A"
        elif "type" in schema_json and schema_json["type"]=="string":
            return "S"
        else:
            return None
    
    def get_object_attr_schema(self,obj=None):
        if obj is not None:
            if "properties" in obj:
                return obj["properties"]
            elif "items" in obj:
                return obj["items"]
            else:
                return obj
            
    def get_key_value(self,schema_json=None, key=None) :
        if schema_json is None:
            schema_json = self.schema_json
        if schema_json is not None and key in schema_json:
            return schema_json[key]
        else:
            return None
    
    def get_object_schema(self,schema_json=None, key=None):
        if schema_json is None:
            schema_json = self.schema_json
        
        val = self.get_key_value(schema_json,key)
        if val is not None:
            if self.get_object_type(val) == "O":                
                return self.get_object_attr_schema(val)
            elif self.get_object_type(val) == "A":
                return self.get_object_attr_schema(val)
            else:
                return None
    
    def is_only_one_attr(self,obj=None):
        if obj is not None:
            if len(obj)==2 and "type" in obj:
                return True
            elif len(obj)==3 and "type" in obj and "required" in obj:
                return True
            else:
                return False
       
    def is_only_one_attr_not_obj(self,obj=None):
        return obj is not None and "type" in obj and obj["type"] not in ["object","array"]
            
    def is_array_node(self,schema_json=None):
        return self.get_object_type(schema_json)=="A"
    
    def is_object_node(self,schema_json=None):
        return self.get_object_type(schema_json)=="O"
    
    def is_single_node_no_child(self,schema_json=None):
        return self.is_only_one_attr_not_obj(schema_json)
    
tmp = {"text": {"type": "string"}}
ijs = JsonSchemaUtil(cde_pth,"fspc.json",None)
title1 = ijs.get_object_schema(None,"title")
print(title1)
title2 = ijs.get_object_schema(jsn,"component")
print("is_only_one_attr : ",ijs.is_only_one_attr(title2))
print("is_array_node : ",ijs.is_array_node(ijs.get_key_value(None,"component")))
print("is_object_node : ",ijs.is_object_node(ijs.get_key_value(None,"title")))
print("is_single_node_no_child : ",ijs.is_single_node_no_child(tmp["text"]))
print(ijs.is_only_one_attr_not_obj(tmp["text"]))


class XmlUtil:
    def __init__(self, file_location=None,file_name=None,bs_xml=None):
        
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.file_name = file_name
        self.bs_xml = None
        self.bs_xml = bs_xml if bs_xml is not None else self.get_xml_parsed(file_location,file_name)
        
    def parse_xml_file(self,file_location=None,file_name=None):
        print("parse_xml_file called")
        if file_location is None:
            file_location = self.file_location
        if file_name is None:
            file_name = self.file_name
            
        bs_xml = BeautifulSoup(open(file_location+'/'+file_name), 'xml')
        print(type(bs_xml))
        return bs_xml
    
    def get_xml_parsed(self,file_location=None,file_name=None):
        if self.bs_xml is None:
            self.bs_xml = self.parse_xml_file(file_location,file_name)
        return self.bs_xml
    
    def find_node_by_name(self, name,bs_xml = None):
        if bs_xml is None:
            bs_xml = self.bs_xml
        if name is not None:
            found_bs = xml_parser.find(name)
            return found_bs
        else:
            return None
    
    def is_attribute_data_key(self,key):
        if len([ch for ch in ["_","__"] if ch in key])>0:
            return True
        else:
            return False
        
    def get_attribute_key(self,key):
        if key is None:
            return None
        if "_" in key:
            return key.replace("_","")
        elif "__" in key:
            return key.replace("__","")
        else:
            return key
    
    def get_node_attribute_value(self, key, bs_xml =None, is_search_node = False, node_name = None):
        if bs_xml is None:
            bs_xml = self.bs_xml
        if is_search_node == True:
            bs_xml = self.find_node_by_name(bs_xml,node_name)
        if bs_xml is not None and key is not None:
            key = self.get_attribute_key(key)
            return bs_xml[key]
        
    def get_node_text_value(self, key, bs_xml = None, is_search_node =False,node_name=None):
        if bs_xml is None:
            bs_xml = self.bs_xml
        if bs_xml is not None and key is not None:
            if is_search_node == True:
                bs_xml = self.find_node_by_name(bs_xml,node_name)
            return bs_xml.contents
        else:
            return None
        
    def get_node_tree(self, key,bs_xml = None):
        if key is not None:
            bs_xml = bs_xml if bs_xml is not None else self.bs_xml
            return bs_xml[key]
        else:
            return None
    
    def get_childrens(self, bs_xml=None):
        bs_xml = bs_xml if bs_xml is not None else self.bs_xml
        if self.hasChild(bs_xml):
            return list(bs_xml.children)
    
    def hasChild(node):
        print(type(node))
        try:
            node.children
            return True
        except:
            return False
        
        
    
xmlutil = XmlUtil(None, "car.xml")


None
is_only_one_attr :  None
is_array_node :  False
is_object_node :  False
is_single_node_no_child :  True
True
parse_xml_file called
<class 'bs4.BeautifulSoup'>


In [6]:
class SchemaToJsonSample:
    def __init__(self, 
                 file_location=None,
                 file_name=None, 
                 xml_file_location=None, 
                 xml_file_path=None, 
                 schema_json=None,
                 bs_xml=None):
        self.schema_json = schema_json
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.file_name = file_name
        
        self.json_util = JsonSchemaUtil(self.file_location,self.file_name,self.schema_json)
        
        self.schema_json = schema_json if schema_json is not None else self.json_util.schema_json
        
        self.xml_file_location = xml_file_location if xml_file_location is not None else os.getcwd()
        self.xml_file_path = xml_file_path
        
        self.xml_util = XmlUtil(xml_file_location,xml_file_path,bs_xml)
        
               
        self.bs_xml = self.xml_util.bs_xml if bs_xml is None else bs_xml
       
        
        #print("------ JsonSchemaUtil initialized with")
        #print(self.file_location)
        #print(self.file_name)        
        #print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        
    def parse_json_schema(self, file_location=None,file_name=None):
        file_location = file_location if file_location is not None else self.file_location
        file_name = file_name if file_name is not None else self.file_name
        
        return self.json_util.load_json(file_location,file_location)
    
    def get_schema_root(self, schema_json = None):
        #print("in Schema Root ")
        schema_json = schema_json if schema_json is not None else self.schema_json
        return self.json_util.get_object_attr_schema(schema_json)
        
    def is_object(self, schema_json = None):
        if schema_json is None:
            schema_json = self.schema_json
        return self.json_util.get_object_type(schema_json) == "O"
        
    def is_array(self, schema_json = None):
        if schema_json is None:
            schema_json = self.schema_json
        return self.json_util.get_object_type(schema_json) == "A"
    
    def clean_key(self,key):
        return self.xml_util.get_attribute_key(key)
    
    def  get_value_from_bs(self,key,bs_xml):
        val = None
        if key is not None:
            if self.xml_util.is_attribute_data_key(key):
                key = self.clean_key(key)
                val = self.xml_util.get_node_attribute_value(key, bs_xml, False, None)
            else:
                val = self.xml_util.get_node_tree(key,bs_xml)
        return val
                
            
        
    '''
        Take an object schema and iterate over its properties.
    '''
    def iterate_object_properties(self,value_json=None, schema_json=None, bs_xml = None, key=None):
        print("iterate_object_properties :", value_json)
        if value_json is None:
            value_json = {}
        if schema_json is None:
            schema_json = self.schema_json
        if bs_xml is None:
            bs_xml = self.bs_xml
            
        return value_json
                
               
        
    
    def travel_schema(self,schema_json = None):
        json_root = self.get_schema_root(schema_json)
        value_json = {}
        #print("JSON ROOT ",json_root)
        value_json= self.iterate_object_properties(value_json, json_root, bs_xml = None, key=None)
                
        return value_json
    
    def sample_json_from_schema(self,schema_json = None ):
        value_json = self.travel_schema()
        return value_json
    
    
    
sjs = SchemaToJsonSample(cde_pth,"fspc.json",cde_pth,"car.xml",None,None)
sample_json1 = sjs.sample_json_from_schema(None)
#print(sample_json)
RenderJSON(sample_json1)

parse_xml_file called
<class 'bs4.BeautifulSoup'>
iterate_object_properties : {}


In [9]:
finalJ ={}
'''for key in jsn:
    print(key)
    if key == "id":        
        xp1 = xml_parser.find(key)
        if "type" in jsn[key]:
            if jsn[key]["type"] =="object":
                finalJ[key]={}
                for k1 in jsn[key]["properties"]:
                    if "_" in k1:
                        key1 = k1[1:]
                        print(":"+key1)
                        finalJ[key][key1] = xp1[key1]
print(finalJ)
'''
counter = 0 
def assign_dict_values(schema_json=None, xml_parse=None,outJson=None, counter=0):
    print("Running Loop ~~~~~~~~~~~~ "+str(counter))
    counter = counter + 1
    if schema_json is not None and xml_parse is not None:
        for schema_key in schema_json:
            print("Running for :: --->"+schema_key)       
            xp1 = xml_parser.find(schema_key)
            print(type(xp1))
            #print(xp1)
            if xp1 is not None and len(xp1.attrs) > 0:
                if "type" in jsn[schema_key] and schema_json[schema_key]["type"] == "object":
                    outJson[schema_key] = {}
                    for pkey in schema_json[schema_key]["properties"]:
                        rkey = pkey
                        if "__" in pkey:
                            rkey = pkey.replace("__" ,"")
                            #print(":1>"+rkey)
                        elif "_" in pkey:
                            rkey = pkey[1:]
                            #print("Going For :>"+rkey)
                        #print(xp1.attrs)
                        if len(xp1.attrs) == 0:
                            print(xp1.contents)
                            if rkey == "title":
                                outJson[schema_key][rkey] =xp1.contents

                        else:
                            outJson[schema_key][rkey] = xp1[rkey]

                elif "type" in jsn[schema_key] and schema_json[schema_key]["type"] == "array":
                    outJson[schema_key] =[]
            else:
                print("Empty Attr :"+schema_key)
                if schema_key == "title":
                    outJson[schema_key]={}
                    outJson[schema_key]["title"] =xp1.contents
                else:                    
                    outJson[schema_key]={}
                    #print(schema_json[schema_key])
                    print("000000000000 :"+ schema_key)
                    if "type" in schema_json[schema_key]:
                        if schema_json[schema_key]["type"] =="object":
                            if "properties" in schema_json[schema_key]:
                                for pkey in schema_json[schema_key]["properties"]:
                                    rkey = pkey
                                    if "__" in pkey:
                                        rkey = pkey.replace("__" ,"")
                                        #print(":1>"+rkey)
                                    elif "_" in pkey:
                                        rkey = pkey[1:]
                                        print("Going For 1 :>"+rkey)
                                    else:
                                        print(rkey)
                                    #print(xp1)
                                    print("-------")
                                    print(pkey)
                                    outJson[schema_key][rkey]={}
                                    assign_dict_values(schema_json[schema_key]["properties"][pkey], 
                                                       xp1,outJson[schema_key][rkey],counter)
                            else:
                                print(" DING DING")
                        elif schema_json[schema_key]["type"] =="array":
                            for pkey in schema_json[schema_key]["items"]:
                                rkey = pkey
                                if "__" in pkey:
                                    rkey = pkey.replace("__" ,"")
                                    #print(":1>"+rkey)
                                elif "_" in pkey:
                                    rkey = pkey[1:]
                                    print("Going For 2 :>"+rkey)
                                else:
                                    print(rkey)
                                print(xp1)
                                outJson[schema_key][rkey]={}
                                assign_dict_values(schema_json[schema_key]["properties"][pkey], 
                                                   xp1,outJson[schema_key][rkey],counter)
                        else:
                            print("FFF DING DING")
                                
                            

                    
                
    return outJson
    
    
value_json = assign_dict_values(jsn,xml_parser,finalJ)
print(value_json) 
#RenderJSON(value_json)

Running Loop ~~~~~~~~~~~~ 0
Running for :: --->$schema
<class 'NoneType'>
Empty Attr :$schema
000000000000 :$schema
Running for :: --->title
<class 'bs4.element.Tag'>
Empty Attr :title
Running for :: --->type
<class 'NoneType'>
Empty Attr :type
000000000000 :type
Running for :: --->properties
<class 'NoneType'>
Empty Attr :properties
000000000000 :properties
Running for :: --->required
<class 'NoneType'>
Empty Attr :required
000000000000 :required
{'$schema': {}, 'title': {'title': ['Hydrocodone Bitartrate and Acetaminophen Tablets, USP', <br/>, ' CII Revised: April 2020', <br/>, '\n']}, 'type': {}, 'properties': {}, 'required': {}}


%3|1684160470.329|FAIL|rdkafka#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)


In [8]:
#!/usr/bin/env python

import sys
from random import choice
from argparse import ArgumentParser, FileType
from configparser import ConfigParser
from confluent_kafka import Producer

class KafkaConnect:
    def __init__(self, file_location=None, config_file_name=None,topic = None) -> None:
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.config_file_name = "kafka-connect.ini" if config_file_name is None else config_file_name
        
        self.config_file = file_location+"/"+config_file_name
        self.topic ="dailymeds" if topic is None else topic
        # Parse the configuration.
        # See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
        config_parser = ConfigParser()
        print("File ini:",self.config_file)
        config_parser.read(self.config_file)
        print(config_parser)
        print(config_parser.has_section('default'))
        config = dict(config_parser['default'])
        print("---- hola config --")
        print(config)
        # Create Producer instance
        self.producer = Producer(config)

    # Optional per-message delivery callback (triggered by poll() or flush())
    # when a message has been successfully delivered or permanently
    # failed delivery (after retries).
    def delivery_callback(self,err, msg):
        if err:
            print('ERROR: Message failed delivery: {}'.format(err))
        else:
            print("Produced event to topic {topic}: key = {key:12} value = {value:12}".format(
                topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))


    def add_grabage(self, topic = None):
        # Produce data by selecting random values from these lists.

        topic = self.topic if topic is None else topic
        
        user_ids = ['eabara', 'jsmith', 'sgarcia', 'jbernard', 'htanaka', 'awalther']
        products = ['book', 'alarm clock', 't-shirts', 'gift card', 'batteries']

        count = 0
        for _ in range(10):

            user_id = choice(user_ids)
            product = choice(products)
            self.producer.produce(topic, product, user_id, callback=self.delivery_callback)
            count += 1

        # Block until the messages are sent.
        self.producer.poll(10000)
        self.producer.flush()

kconnect = KafkaConnect(cde_pth,"kafka-connect.ini")
print(type(kconnect.producer))

File ini: /Users/anand/Documents/sayansi/workspaces/condaws/dailymed_sracping/kafka-connect.ini
True
---- hola config --
{'bootstrap.servers': 'localhost:9092'}
<class 'cimpl.Producer'>


%3|1684160423.248|FAIL|rdkafka#producer-1| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)


In [ ]:
#!/usr/bin/env python

import sys
from argparse import ArgumentParser, FileType
from configparser import ConfigParser
from confluent_kafka import Consumer, OFFSET_BEGINNING

class KafkaConsumer:
    def __init__(self, file_location=None, config_file_name=None,topic =None) -> None:
        self.file_location = os.getcwd() if file_location is None  else file_location
        self.config_file_name = "kafka-connect.ini" if config_file_name is None else config_file_name
        
        self.config_file = file_location+"/"+config_file_name
        self.topic = "dailymeds" if topic is None else topic

        # Parse the configuration.
        # See https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md

        config_parser = ConfigParser()
        
        print("File ini:",self.config_file)
        config_parser.read(self.config_file)
        print(config_parser)
        print(config_parser.has_section('default'))
        config = dict(config_parser['default'])
        print("---- hola config --")
       
        print(config)
        config.update(config_parser['consumer'])
        print(config)
        self.config = config
        # Create Consumer instance
        self.consumer = Consumer(config)

    # Set up a callback to handle the '--reset' flag.
    def reset_offset(self,consumer, partitions):
        print(self.config['auto.offset.reset'])
        if self.config['auto.offset.reset']:
            for p in partitions:
                p.offset = OFFSET_BEGINNING
            consumer.assign(partitions)

    def subscribe_topic(self,topic=None):
        # Subscribe to topic
        topic = self.topic if topic is None else topic
        print(topic)
        self.consumer.subscribe([topic], on_assign=self.reset_offset)

    def poll_print(self,jjo=None):
        # Poll for new messages from Kafka and print them.
        try:
            while True:
                msg = self.consumer.poll(1.0)
                if msg is None:
                    # Initial message consumption may take up to
                    # `session.timeout.ms` for the consumer group to
                    # rebalance and start consuming
                    print("Waiting...")
                elif msg.error():
                    print("ERROR: %s".format(msg.error()))
                else:
                    # Extract the (optional) key and value, and print.

                    print("Consumed event from topic {topic}: key = {key:12} value = {value:12}".format(
                        topic=msg.topic(), key=msg.key().decode('utf-8'), value=msg.value().decode('utf-8')))
        except KeyboardInterrupt:
            pass
        finally:
            # Leave group and commit final offsets
            self.consumer.close()


kconsumer = KafkaConsumer(cde_pth,"kafka-connect.ini","dailymeds")
kconsumer.subscribe_topic()
print(type(kconsumer))
#kconsumer.consumer.close()

In [ ]:
#kconnect.add_grabage()

In [ ]:
#kconsumer.poll_print()

In [11]:
import xmltodict
import json

class XmlToDictUtil:
    def __init__(self,file_path=None,file_name=None):
        self.json_data = None
        self.file_path = file_path
        self.file_name = file_name
        pass
    
    def read_xml_file(self,file_path=None,file_name=None):
        file_location = file_path +"/"+file_name
        print(file_location)
        with open(file_location, 'r') as file:
            xml_data = file.read()
            ordered_dict = xmltodict.parse(xml_data)
            self.json_data = json.dumps(ordered_dict)
        return self.json_data



In [12]:
xdu = XmlToDictUtil(cde_pth,"response.xml")
jd = xdu.read_xml_file(cde_pth,"response.xml")

/Users/anand/Documents/sayansi/workspaces/condaws/dailymed_sracping/response.xml


In [12]:
print(jd)

{"document": {"@xmlns": "urn:hl7-org:v3", "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance", "@xsi:schemaLocation": "urn:hl7-org:v3 https://www.accessdata.fda.gov/spl/schema/spl.xsd", "id": {"@root": "490c0526-0369-40d6-85b1-9e037655bba0"}, "code": {"@code": "34391-3", "@codeSystem": "2.16.840.1.113883.6.1", "@displayName": "HUMAN PRESCRIPTION DRUG LABEL"}, "title": {"br": [null, null, null], "#text": "These highlights do not include all the information needed to use ESCITALOPRAM TABLETS safely and effectively. See full prescribing information for ESCITALOPRAM TABLETS.\n      ESCITALOPRAM tablets, for oral useInitial U.S. Approval: 2002"}, "effectiveTime": {"@value": "20230414"}, "setId": {"@root": "155e973a-f4ff-4e5a-b937-09f1e106fe39"}, "versionNumber": {"@value": "10"}, "author": {"time": null, "assignedEntity": {"representedOrganization": {"id": {"@extension": "828343017", "@root": "1.3.6.1.4.1.519.1"}, "name": "SOLCO HEALTHCARE US, LLC"}}}, "component": {"structuredBody": 